This notebook takes Luminescence, Fluorescence and background intensity measurements made in Fiji/ImageJ for an image file. It subtracts the background from respective channels, calculates L/F ratios, applied pH correction and calculates ATP concentration using user defined Km and Vmax.

In [ ]:
# import modules
import os
import pandas as pd
import numpy as np
from statistics import mean
import random
import re

# Get the list of all files and directories
Address = r"folder_path_to_image_file_and_Fiji_Macro_output"
Address = Address.replace("\\","/") + "/"


list_files =  os.listdir(Address)
list_files.sort()
#Get values of each spine from respective filenames
df_output = pd.DataFrame()
for files in list_files:
    if files.endswith(".csv"):
        filename = files.split(".")[0]
        df = pd.read_csv(Address+files)
        df_output[filename] = df['Mean']

# Get background values per frame for F and L
BGAddress = Address + "bg/"
list_files =  os.listdir(BGAddress)
list_files.sort()
df_bg = pd.DataFrame()
for files in list_files:
    if files.endswith(".csv"):
        filename = files.split(".")[0]
        df = pd.read_csv(BGAddress+files)
        df_bg[filename] = df['Average']


#Make list of column names and split into two lists of L and F names    
cols = df_output.columns.tolist()
pairs = int(len(cols)/2)
F_cols = cols[0: pairs]
L_cols = cols[pairs:]
#Rearrange L and F names in list to make pairs
re_cols = []
for i in range(0,pairs):
    re_cols.append(L_cols[i])
    re_cols.append(F_cols[i])
#Rearrange columns in dataframe using rearranged list of names
df_output  = df_output[re_cols]

#Subtract background intensities
for F_spine in F_cols:
    df_output[F_spine] -=  df_bg["F BG AVG"]
    
for L_spine in L_cols:
    df_output[L_spine] -= df_bg["L BG AVG"]

#Ask user for saving name
output_filename = input("Save Raw Values As (default: All_bgs): ") or "All_bgs" + ".xlsx"
#Save Raw values as Excel Sheet
df_output.to_excel(Address+output_filename, sheet_name='Raw', index=False)
print("Raw Values for All Spines Merged and Saved as:")
print(Address+output_filename)

#Change Excel File Name for different files
Excel_file_name = output_filename
#Redefine Location with the filename
Location = Address + Excel_file_name 

#Time Points for Base
base_t = 3 #for plasticity induction experiments or 10 for resting state experiments


''' #uncomment for adding laser power correction option
# Asks if Laser Power Correction is needed
lpc = input("Apply Laser Power Correction  (y/n) ? ")

if lpc == "y":
    lpcf = 1.7
else:
    lpcf = 1.0
'''
lpcf = 1.0
#No laser power correction was applied for the datasets in this manuscript

# Asks if Apparent Photobleaching Correction is needed
pbc_check = input("Correct for Apparent Photobleaching (y/n) ? ")
#No photobleach correction was applied for the datasets in this manuscript

if pbc_check == "y":
    #Set Apparent Photobleach Correction factors 
    pbc1 = 0.96
    pbc2 = 0.89
    pbc3 = 0.79
    pbc4 = 0.75
    
else:
    #Set Apparent Photobleach Correction factors to 1 i.e. No Correction
    pbc1 = 1
    pbc2 = 1
    pbc3 = 1
    pbc4 = 1

#Opens Excel File and makes list of sheets
with pd.ExcelFile(Location) as xlsx:
    sheet_ls = xlsx.sheet_names
    print("List of Sheets in the Excel File:")
    for sheet in sheet_ls:
        print(sheet_ls.index(sheet), sheet)

#Ask for list of sheets to use for Output      
selection_ls = input("Make Comma Separated List of Numbers Corresponding Sheet Name ")
selection_ls = selection_ls.split(",")
selected_sheet_ls = []
for selection in selection_ls:
    selected_sheet_ls.append(sheet_ls[int(selection)])
print("List of Sheets Selected: ",selected_sheet_ls) #prints selected sheets' list for cross-checking

#Run Loop to Process Each Sheet
for sheet in selected_sheet_ls:
    sheet_name = sheet
    print("Processing Sheet: ", sheet_name)

    #Open Excel File and given sheet
    with pd.ExcelFile(Location) as xlsx:
        df = pd.read_excel(xlsx, sheet_name, index_col=None)
        sp_id_ls = []
        for col in list(df.columns):
            if "Spine_" in col:
                start = col.find('Spine_')
                sp_id = int(re.search(r'\d+', col[start+2:]).group())
                if sp_id not in sp_id_ls:
                    sp_id_ls.append(sp_id)

        print("list of spines: " , sp_id_ls) #prints list of spines in sheet

        df_output = pd.DataFrame()
        for sp_id in sp_id_ls:
            F = "F_Raw_Spine_" + str(sp_id) #Change " " to fluorescence prefix of spine numbers
            L = "L_Raw_Spine_" + str(sp_id) #Change " " to luminescence prefix of spine numbers

            F_raw_ls = list(df[F][df[F] != 0].dropna())
            L_raw_ls = list(df[L][df[L] != 0].dropna())
            #print(len(F_raw_ls))
            #print(F_raw_ls)
            
            #Photobleach Correction
            F_pbc_ls = []
            for F_raw in F_raw_ls:
                i = F_raw_ls.index(F_raw)
                if i >= 0 and i < 3:
                    F_pbc = F_raw
                elif i >= 3 and i < 20:
                    F_pbc = F_raw/pbc1
                elif i >= 3 and i < 30:
                    F_pbc = F_raw/pbc2
                elif i >= 3 and i < 40:
                    F_pbc = F_raw/pbc3
                elif i >= 3 and i < 50:
                    F_pbc = F_raw/pbc4

                F_pbc_ls.append(F_pbc)    
            #print(len(F_pbc_ls))
            #print(F_pbc_ls)
            
            
            #Intracellular pH Calculation
            F_ref = mean(F_pbc_ls[:base_t])
            #print(F_ref)
            pKaF = 6.5
            pH_ref = 6.9
            pHi_ls = []
            for F_pbc in F_pbc_ls:
                i = F_pbc_ls.index(F_pbc)
                if i >= 0 and i < base_t:
                    pHi = pH_ref
                else:
                    pHi = pKaF - np.log10( ((1 + 10**(pKaF - pH_ref)) / (F_pbc/F_ref))  - 1)

                pHi_ls.append(pHi)

            #print(len(pHi_ls))
            #print(pHi_ls)

            #The pH correction for F has been done to calculate F at pH 7
            #pH Corrected F Calculation; All values should be same i.e. F_total
            F_pHc_ls = []
            for pHi in pHi_ls:
                i = pHi_ls.index(pHi)
                if i >= 0 and i < base_t:
                    i = i + base_t
                    pHi = pHi_ls[i]
                    F_pbc = F_pbc_ls[i]
                    F_pHc = lpcf * F_pbc *  (  (1+10**(pKaF-pHi))  /  (1+10**(pKaF-7))  )
                else:
                    F_pbc = F_pbc_ls[i]
                    F_pHc = lpcf * F_pbc *  (  (1+10**(pKaF-pHi))  /  (1+10**(pKaF-7))  )
                F_pHc_ls.append(F_pHc)

            #print(len(F_pHc_ls))
            #print(F_pHc_ls)
            
            #L by F Ratio Calculation
            LbyF_ls =[]
            F_total = random.choice(F_pHc_ls)
            for L_raw in L_raw_ls:
                i = L_raw_ls.index(L_raw)
                LbyF = L_raw / F_total

                LbyF_ls.append(LbyF)

            #print(len(LbyF_ls))
            #print(LbyF_ls)
            
            #ATP concentration Calculation
            pKaATP = 7.03 
            pH7 = 7.0
            Km = 1.446 #for Calibration 2 or 1.501 for Calibration 1
            Vmax = 1.649  #for Calibration 2 or 0.973 for Calibration 1
            ATP_ls = []
            for LbyF in LbyF_ls:
                i = LbyF_ls.index(LbyF)
                pHi = pHi_ls[i]
                ATP_pH7 = (Km * (  (1+10**(pKaATP-pHi))/(1+10**(pKaATP-pH7))  ) * LbyF) / (Vmax - LbyF)
                ATP_ls.append(ATP_pH7)

            #print(len(ATP_ls))
            #print(ATP_ls)

            suffix = "_Spine_"+ str(sp_id)
            df_output['L_raw' + suffix] = pd.Series(L_raw_ls)
            df_output['F_raw' + suffix] = pd.Series(F_raw_ls)
            df_output['F_pbc' + suffix] = pd.Series(F_pbc_ls)
            df_output['pHi' + suffix] = pd.Series(pHi_ls)
            df_output['F_pHc' + suffix] = pd.Series(F_pHc_ls)
            df_output['LbyF_pHc' + suffix] = pd.Series(LbyF_ls)
            df_output['ATP(mM)' + suffix] = pd.Series(ATP_ls)
            
        output_filename = input("Save Calculated Values As (default: All_ATP): ") or "All_ATP" + ".xlsx"
        df_output.to_excel(Address+output_filename, sheet_name='Raw', index=False)